# MLDM Monday - Introduction to Julia

<h2>
<font color='black'>
Yin-Chen Liao (Dboy)
</font>
</h2>

# Is Julia Object-Oriented, Functional or What?

我想在座的各位身為 R, Python 或其他許多主流語言的使用者，

對於 OOP 或 Functional Programming 應該都不陌生，

在接觸 Julia 這個新語言的時候，

想必很自然地會問這樣的問題: Julia 到底是個怎樣的語言?


### 引自 Julia [官方文件](http://julia.readthedocs.org/en/latest/manual/introduction/):

<pre>
Julia features optional typing, multiple dispatch, and good performance, achieved using type inference and just-in-time (JIT) compilation, implemented using LLVM. <strong>It is multi-paradigm, combining features of imperative, functional, and object-oriented programming.</strong>
</pre>

所以我想，如果 Julia 是活生生的動物的話，

大概長這樣吧....


<center>
<img src=http://163.20.173.57/stud96/919/33/p2/1.jpg />
</center>



因此會著重在下面幾個重點:

1. Julia 的 Type System
2. Methods
3. 聽說 Julia 很快? 
  - Loops in Julia: 以碎形為例。(Gadfly 圖超美的!)
  - Recursion in Julia: 萬年經典例子 -- 費式數列
  - 實作 Simple Neural Network: Julia v.s Python(Numpy)
4. 如何學習 Julia:
  - Document 。
  - 看看 Julia 原始碼。例如 base.jl --> 重要的 Julia 核心 code

有時間的機會的話....

 - Parallel Computation in Julia (CPU-bound)
 - Socket Programming in Julia

## Type System

<pre>
Describing Julia in the lingo of type systems, <strong>it is: dynamic, nominative and parametric.</strong> Generic types can be parameterized, and the hierarchical relationships between types are explicitly declared, rather than implied by compatible structure. 
</pre>

- Abstract Type
- Concrete Type
- Composite Type (常用于自定義資料結構 --> 類似 C 的 struct)
- Parametric Type (類似 C++ 的 Template )

嗯....我知道有說等於沒說

我們來看一些例子。

In [1]:
# 自定義 "男人" 類別 
type Man
    gender::String       # 屬性 (attribute):: 資料形態
    first_name::String
    last_name::String
    
    function Man(fn::String, ln::String)
        gender = "Male"
        new(gender, fn, ln)
    end
end

In [2]:
type Woman
    gender::String
    first_name::String
    last_name::String
    
    function Woman(fn::String, ln::String)
        gender = "Female"
        new(gender, fn, ln)
    end
end

In [7]:
# String Interpolation
me = Man("Yin-Chen", "Liao");
println("Man:")
println("My first name is $(me.first_name).")
println("My last name is ", me.last_name, '.')
println()

girl_friend = Woman("Donot", "Exists")
println("Woman:")
println("I, $(girl_friend.first_name) $(girl_friend.last_name),", " am $(me.first_name)'s girl friend.")

Man:
My first name is Yin-Chen.
My last name is Liao.

Woman:
I, Donot Exists, am Yin-Chen's girl friend.


In [13]:
# 自定義 + 法
function +(man::Man, woman::Woman)
    ind = rand(1:2)
    child_first_name = woman.first_name
    child_last_name = man.last_name
    if ind == 2
        child_first_name, child_last_name = man.first_name, woman.last_name
        return Woman(child_first_name, child_last_name)
    end
    return Man(child_first_name, child_last_name)
end

+(woman::Woman, man::Man) = +(man, woman)

+ (generic function with 121 methods)

In [14]:
me + girl_friend

Man("Male","Donot","Liao")

In [15]:
me + girl_friend

Woman("Female","Yin-Chen","Exists")

### 用行動支持多元成家!

誰說只有 "男人 + 女人" 或者 "女人 + 男人" !?

只要有愛，人人皆可成家。


但懶惰如我或工程師心中獨白: 要定義 4 個 `+` 好麻煩喔....。

用 `Abstract Type` 簡化 code:

In [1]:
# 定義一個 abstract type
abstract Human 

type Man <: Human  # 簡單加上這一行
    gender::String
    first_name::String
    last_name::String
    
    function Man(fn::String, ln::String)
        gender = "Male"
        new(gender, fn, ln)
    end
end

type Woman <: Human # 簡單加上這一行
    gender::String
    first_name::String
    last_name::String
    
    function Woman(fn::String, ln::String)
        gender = "Female"
        new(gender, fn, ln)
    end
end

In [16]:
# 只要定義一次 + 就可以用在所有男人跟女人的組合上。
function +(h1::Human, h2::Human)  
    child_gender = h1.gender
    child_first_name = h2.first_name
    child_last_name = h1.last_name
    ind1 = rand(1:2)
    ind2 = rand(1:2)
    if ind2 == 2
        child_first_name, child_last_name = h1.first_name, h2.last_name
    end
    if ind1 == 2
        child_gender = h2.gender
    end
    if child_gender == "Male"
        return Man(child_first_name, child_last_name)
    else
        return Woman(child_first_name, child_last_name)
    end
end

+ (generic function with 121 methods)

In [17]:
# 如果我不小心毀了你對 Snoopy 的印象....I'm sorry.

charlie = Man("Charlie", "Brown")
linusvan = Man("Linusvan", "Pelt")
lucy = Woman("Lucy", "Pelt")
sally = Woman("Sally", "Brown")

println("Charlie and Lucy have a child: ", charlie + lucy)
println("Charlie and Linusvan adopt a child: ", charlie + linusvan)
println("Sally and Lucy adopt another child: ", sally + lucy)

Charlie and Lucy have a child: Man("Male","Lucy","Brown")
Charlie and Linusvan adopt a child: Man("Male","Linusvan","Brown")
Sally and Lucy adopt another child: Woman("Female","Sally","Pelt")


## Parametric Type

- 在 Julia 裡，可以為 type 加上 parameter 。
- 很抱歉我沒想到什麼通俗的例子
  - 關於這個主題我會用線性代數裡的 Vector space 來做例子


試想你想要在操作一個 2 維的向量空間，

同樣是 2 為向量空間可以有不同的 field 。

舉例來說 $\mathbb{R}^2$, $\mathbb{C}^2$ ...etc。

那你可能會這樣做:

In [1]:
# Vector space over R^2
type VectorOverR
    x::Float64
    y::Float64
end

# Vector space over C^2
type VectorOverC
    x::Complex{Float64}
    y::Complex{Float64}
end

v_r = VectorOverR(1.1, 2.3);
v_c = VectorOverC(1. + 3.im, 2.1 + 3.5im);

In [2]:
println("v_r: ", v_r)
println("v_c: ", v_c)

v_r: VectorOverR(1.1,2.3)
v_c: VectorOverC(1.0 + 3.0im,2.1 + 3.5im)


OK....我知道這會動，但是....

- 難不成我有 10 種 Field 我就要寫 10 種 Vector space!?
- 這些 code 都長好像喔，我一定要浪費時間在這種重複性的 code 嗎?
- 你不知道 python 有個很好的準則叫做 DRY 嗎? (Donot Repeat Yourself)
- 我不想寫這種 code !!(吼)


用 parametric type 去簡化 code:


In [3]:
type VectorSpace{T}
    x::T
    y::T
end

v_r = VectorSpace{Float64}(1.1, 2.3);
v_c = VectorSpace{Complex{Float64}}(1., 2.);

In [4]:
println("v_r: ", v_r)
println("v_c: ", v_c)

v_r: VectorSpace{Float64}(1.1,2.3)
v_c: VectorSpace{Complex{Float64}}(1.0 + 0.0im,2.0 + 0.0im)


## References

- [Julia 官方文件](http://julia.readthedocs.org/en/latest/)
- [JIT (Just-In-Time) compiler](http://stackoverflow.com/questions/95635/what-does-a-just-in-time-jit-compiler-do)  [StackOverflow]
- [LLVM](http://aosabook.org/en/llvm.html)

<div class='reveal'>
 <center>
 <font size=16> Thanks for your attention </font>
 </center>
</div>
<br>
<br>
<div class='reveal'>
  <div class='fragment'>
  <center>
    <font size=10 color=blue>Q/A</font>
  </center>
  </div>
</div>

## Typos Found in the Doc

- [C Interface](http://julia.readthedocs.org/en/latest/manual/calling-c-and-fortran-code/)

In [31]:
# Pointer type in Julia
Ptr

Ptr{T}

In [49]:
function getenv(var::String)
  val = ccall((:getenv, "libc"),
    Ptr{Uint8}, (Ptr{Uint8},), var)
  if val == C_NULL
    error("getenv: undefined variable: ", var)
  end
  bytestring(val)
end

getenv (generic function with 1 method)

In [50]:
getenv("SHELL")

"/bin/bash"